In [2]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import clear_output as clear
from ipywidgets import IntProgress
from IPython.display import display
from Slide import * 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [193]:
max_num = 10
slides=[]
p_bar = IntProgress(max=max_num)
p_bar.value = 0
p_bar.description = 'loading'
# load image information
with open('data/image_info.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        s_name = row[1].strip()
        s_image = mpimg.imread('data/ImageDataset/{image_name}.jpg'.format(image_name = s_name))
        s_valence = float(row[4])
        s_arousal = float(row[5])
        s_intensity = float(row[6])
        s_dominant_color = np.array(list(map(float,row[7:10])))
        slides.append(Slide(p_bar.value,
                            s_name,
                            s_image,
                            s_valence,
                            s_arousal,
                            s_intensity,
                            s_dominant_color))
        p_bar.value += 1
        if p_bar.value >= max_num:
            break
        p_bar.description = '{loaded}-{percent}%'.format(loaded=p_bar.value+1,percent=(p_bar.value+1)/max_num*100)
        display(p_bar)
      


IntProgress(value=1, description='1-10.0%', max=10)

IntProgress(value=2, description='2-20.0%', max=10)

IntProgress(value=3, description='3-30.0%', max=10)

IntProgress(value=4, description='4-40.0%', max=10)

IntProgress(value=5, description='5-50.0%', max=10)

IntProgress(value=6, description='6-60.0%', max=10)

IntProgress(value=7, description='7-70.0%', max=10)

IntProgress(value=8, description='8-80.0%', max=10)

IntProgress(value=9, description='9-90.0%', max=10)

In [189]:
from Slide import * 

# load target curve information
curve_info_mat = []
counter = 0
with open('data/curve_info.csv') as csvfile:
    csv_reader = csv.reader(csvfile,delimiter = ',')
    for row in csv_reader:
        curve_info_mat.append(list(map(float,row)))
target_curves = TargetCurves(curve_info_mat)


7


In [194]:
print(slides[0].id)
print(slides[100].id)

0


IndexError: list index out of range

In [205]:
def cal_attribute_cost(slide_show:List,target_values:List,weights:List):
    eps = 0.0000000001
    # valence, arousal, intensity
    value_mat = np.array([[slide.valence, slide.arousal,slide.intensity, *slide.dominant_color] for slide in slide_show])
    tmp_values = np.mean(value_mat,axis=0)
    initial_costs = 1-np.exp( -np.power(tmp_values- target_values,2)/(eps + 2*np.power(target_values,2)))
    weighted_costs = initial_costs * weights
    vaic_total_costs = np.sum(weighted_costs)
    return vaic_total_costs
   
def cal_sequence_cost(slide_show,curves,weights):
    va_values = np.transpose(np.array([[slide.valence, slide.arousal] for slide in slide_show]))
    costs = np.sum(np.power(va_values/va_values.max(axis=1)[:,None]-curves,2),axis=1)/10
    return np.sum(costs * weights)
def check_repetition(slide_show):
    indices = [slide.id for slide in slide_show]
    return len(indices) == len(set(indices))

In [206]:
print(len(slides))
A  = check_repetition(slides) * 1.0

10


In [207]:
target_dict = {
    'valence': 0.5,
    'arousal': 0.5,
    'intensity': 0.5,
    'R':0.5,
    'G':0.3,
    'B':0.4,
    'valence_curve':target_curves.straight,
    'arousal_curve':target_curves.straight_curve_hill,
}
weights_dict= {
    'valence': 0.5,
    'arousal': 0.5,
    'intensity': 0.5,
    'R':0.5,
    'G':0.5,
    'B':0.5,
    'valence_curve':0.1,
    'arousal_curve':0.1,
    'repetition':1.0
}
target_values = [target_dict['valence'],
                 target_dict['arousal'],
                 target_dict['intensity'],
                 target_dict['R'],
                 target_dict['G'],
                 target_dict['B']]
attri_weights =  [weights_dict['valence'],
            weights_dict['arousal'],
            weights_dict['intensity'],
            weights_dict['R'],
            weights_dict['G'],
            weights_dict['B']]
curves = np.array([target_dict['valence_curve'],target_dict['arousal_curve']])
curve_weights = np.array([weights_dict['valence_curve'],weights_dict['arousal_curve']])
slideshow = slides[0:10]#np.random.choice(slides,size = 2,replace = True)

print(total_cost)


1.1453552979312276


In [ ]:
MC_config = {
    'replace':[0.5,0.5,0.5,0.5,0.5,0.75,0.75,0.75,0.75,0.75] # update each 100 iteration
    'bad_move':
}
max_iteration = 1000
total_cost = 100000000000000000000
for i in range(max_iteration):
    slideshow = np.random.choice(slides,size = 10,replace = True)
    attri_c = cal_attribute_cost(slideshow,target_values,attri_weights)
    curve_c = cal_sequence_cost(slideshow,curves,curve_weights)
    total_cost = attri_c + curve_c + weights_dict['repetition'] * check_repetition(slides)
plt.figure(figsize=(15,5))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(slideshow[i].image)
    plt.axis('off') # 不显示坐标轴


plt.show()